In [1]:
import quippy
import einops

In [2]:
from quippy.descriptors import Descriptor

desc = Descriptor("soap l_max=6 n_max=12 cutoff=5.0 atom_sigma=0.5")

In [3]:
from moldiff.utils import initialize_mol

In [4]:
mol = initialize_mol("C6H6")

In [5]:
desc.calc(mol, grad=True)

{'has_grad_data': [array([   59911569,           0,   670562216, -1129911336,           1,
                   1,           1,           1,           1,           1,
                   1,           1], dtype=int32),
  array([   60077657,           0,   670562216, -1129911336,           1,
                   1,           1,           1,           1,           1,
                   1,           1], dtype=int32),
  array([   60240848,           0,   670562216, -1129911336,           1,
                   1,           1,           1,           1,           1,
                   1,           1], dtype=int32),
  array([2359295,  -65505,  -65513,      -1,      -1,      -1, 2818109,
              -1,      -1,  -65519,      -1,      -1], dtype=int32),
  array([60569712,        0,        1,        1,        1,        1,
                1,        1,        1,        1,        1,        1],
        dtype=int32),
  array([60703911,        0,        1,        1,        1,        1,
                1,

In [6]:
grad = desc.calc(mol, grad=True)["grad_data"]

In [47]:
desc.calc(test_mol, grad=True)["grad_index_0based"].shape

(82, 2)

In [44]:
desc.calc(test_mol, grad=True).keys()

dict_keys(['has_grad_data', 'ii', 'pos', 'grad_covariance_cutoff', 'covariance_cutoff', 'data', 'has_data', 'grad_data', 'ci', 'grad_index_0based'])

In [7]:
embedding_gradients = einops.rearrange(
            grad,
    "(dummy num_atoms) space embed_dim -> dummy num_atoms space embed_dim",
    num_atoms=12,
)
embedding_gradients = einops.reduce(
    embedding_gradients,
    "num_atoms dummy space embed_dim -> num_atoms space embed_dim",
    "sum",
)

In [8]:
embedding_gradients.shape

(12, 3, 547)

In [9]:
einops.rearrange(grad, "(x num_atoms) space embed_dim -> x num_atoms space embed_dim", num_atoms=12)

array([[[[ 0.00000000e+00, -8.67361738e-19,  0.00000000e+00, ...,
           0.00000000e+00,  4.13590306e-24,  0.00000000e+00],
         [ 2.43853301e-01,  1.76124458e-02,  3.24554721e-03, ...,
          -2.16045145e-07, -1.33858690e-07,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

        [[-2.14168929e-02, -3.70750808e-04,  3.23767576e-04, ...,
          -2.46205141e-08,  3.51553020e-09,  0.00000000e+00],
         [ 1.85688915e-03,  1.89672426e-04,  6.09058800e-05, ...,
          -3.41803996e-09, -3.55872937e-10,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

        [[-1.51165609e-02, -8.18850699e-05, -2.21638206e-04, ...,
           8.56076238e-08,  6.44356224e-08,  0.00000000e+00],
         [-1.68208236e-02, -9.13330559e-05, -2.46350394e-04, ...,
           1.09773489e-07,  6.2047

In [10]:
grad.shape

(144, 3, 547)

In [11]:
144/12

12.0

In [12]:
desc.calc(mol, grad=True)["data"].shape

(12, 547)

In [13]:
from moldiff.sampling import SOAPSimilarityModel

/home/rokas/miniconda3/envs/genesis/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
training_data = [initialize_mol("C6H6"), initialize_mol("CH4")]

In [15]:
scorer = SOAPSimilarityModel(training_data=training_data)

In [16]:
scorer.reference_embeddings.shape

(17, 547)

In [20]:
import numpy as np

In [46]:
test_mol = initialize_mol("C6H6")
test_mol.set_positions(
    2*np.random.normal(size=test_mol.get_positions().shape)
)

In [31]:
112/12

9.333333333333334

In [37]:
scorer(test_mol,t=0)

EinopsError:  Error while processing rearrange-reduction pattern "(dummy num_atoms) space embed_dim -> dummy num_atoms space embed_dim".
 Input tensor shape: (134, 3, 547). Additional info: {'num_atoms': 11}.
 Shape mismatch, can't divide axis of length 134 in chunks of 11

In [35]:
scorer(test_mol,t=0)

array([[ 1.20322861e-19, -1.80792117e-18,  0.00000000e+00],
       [ 1.24413670e-18, -1.14850924e-18,  0.00000000e+00],
       [ 6.10800046e-18, -1.69855879e-19,  0.00000000e+00],
       [ 1.31016386e-18, -2.31316836e-18,  0.00000000e+00],
       [-5.50281176e-19,  5.19722854e-19,  0.00000000e+00],
       [ 2.42182548e-18, -4.92144678e-18,  0.00000000e+00],
       [-4.83203093e-19,  2.78449386e-18,  0.00000000e+00],
       [ 1.86183045e-18,  3.65539871e-18,  0.00000000e+00],
       [ 7.07374871e-19,  1.22756735e-19,  0.00000000e+00],
       [-6.97179718e-19,  3.03529235e-18,  0.00000000e+00],
       [ 2.75566199e-19,  4.06485328e-18,  0.00000000e+00],
       [ 1.33603792e-18, -1.68075304e-18,  0.00000000e+00]])